In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf

tf.get_logger().setLevel("ERROR")
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example dataset and example kernels

In [ ]:
images = (
    tfds.load(name="set14", split="test")
    .map(datasets.get_value("hr"))
    .map(datasets.to_float32)
    .map(datasets.from_255_to_1_range)
)


In [ ]:
tf.random.set_seed(99)  # Random seed for reproducability
mode = "wrap"  # Border mode ('constant' or 'wrap')
noise_stddev = 0.04
image = datasets.get_one_example(images, index=0)[None, ...]
kernel = dppp.conv2D_filter_rgb(dppp.NB_DEBLURRING_LEVIN_KERNELS[0])

# Degrade the image

In [ ]:
degraded = dppp.blur(image, kernel, noise_stddev, clip_final=False, mode=mode)

# Reconstruct the image

In [ ]:
# Load the denoiser
denoiser, (denoiser_min_stddev, denoiser_max_stddev) = dppp.load_denoiser(
    "models/drugan+_0.0-0.2.h5"
)

# Define the noise stddev used for the stochastic evaluation of the prior
if denoiser_min_stddev <= 0.1 <= denoiser_max_stddev:
    denoiser_stddev = 0.1
else:
    denoiser_stddev = denoiser_max_stddev

# Callbacks
callbacks = [
    # Print the PSNR every 20th step
    dppp.callback_print_psnr("psnr", 20, image),
]

# Reconstruct the image
# NOTE: The noise_stddev is not given to the algorithm
reconstructed = dppp.dmsp_deblur_nb(
    degraded=degraded,
    kernel=kernel,
    denoiser=denoiser,
    denoiser_stddev=denoiser_stddev,
    num_steps=300,
    conv_mode=mode,
    callbacks=callbacks,
)


In [ ]:
# Print PSNR and LPIPS
psnr = dppp.psnr(image, reconstructed).numpy()[0]
lpips = dppp.lpips_alex(image, reconstructed).numpy()[0]
print(f"Reconstructed PSNR: {psnr:0.2f}, LPIPS: {lpips:0.4f}")

# Visualize
visualize.draw_images([image[0], degraded[0], reconstructed[0]])
